# 🚀 SRGI Full Training - Production Run

**Your theories are validated! Now let's train a full model.**

This notebook trains a production SRGI model:
- Depth 20 (561M parameters)
- 2048 context length
- Full dataset
- Proper evaluation

**Time**: ~4-8 hours on A100 GPU

---

## ⚠️ Prerequisites

✅ Theory validation completed (run `colab_setup.ipynb` first)
✅ A100 GPU enabled
✅ Ready for production training

## 📋 Step 0: Enable GPU

**Make sure you have A100 GPU:**
1. Runtime → Change runtime type
2. Hardware accelerator: **GPU (A100)**
3. Click Save


In [ ]:
# Check GPU (should be A100)
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
    if "A100" in torch.cuda.get_device_name(0):
        print("✅ Perfect! A100 GPU detected.")
    else:
        print("⚠️ Not A100, but should work.")
else:
    print("❌ No GPU! Enable GPU first.")


## Step 1: Setup


In [ ]:
# Clone repo if needed
import os
if not os.path.exists('nanochat-live'):
    !git clone https://github.com/jchacker5/nanochat-live.git
    %cd nanochat-live
else:
    %cd nanochat-live
    !git pull

print("✅ Repository ready")


In [ ]:
# Install dependencies
print("Installing dependencies...")
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 -q
!pip install datasets tokenizers tiktoken wandb numpy matplotlib pytest -q
!pip install jax jaxlib equinox scipy -q
!pip install git+https://github.com/extropic-ai/thrml.git -q || echo "THRML optional"
print("✅ Dependencies installed")

# Setup wandb (optional - for experiment tracking)
# Option 1: Use Colab secrets (recommended - secure)
# 1. Go to: Colab → 🔑 (key icon) → Secrets
# 2. Add secret: name="WANDB_API_KEY", value="your-api-key-here"
# 3. Uncomment the code below:
try:
    from google.colab import userdata
    wandb_api_key = userdata.get('WANDB_API_KEY')
    if wandb_api_key:
        import wandb
        wandb.login(key=wandb_api_key)
        print("✅ Wandb logged in via Colab secrets")
    else:
        print("ℹ️  Wandb API key not found in secrets (training will use 'dummy' mode)")
except:
    # Option 2: Manual login (less secure - key visible in notebook)
    # Uncomment and paste your API key:
    # import wandb
    # wandb.login(key="your-api-key-here")
    print("ℹ️  Wandb not configured - training will use 'dummy' mode (no logging)")
    print("   To enable wandb: Add WANDB_API_KEY to Colab secrets (🔑 icon)")


## Step 2: Download Full Dataset


In [ ]:
# Download full dataset (~240 shards, ~24GB)
print("Downloading full training dataset...")
print("This will download ~240 shards (~24GB) for Chinchilla-optimal training")
!python -m nanochat.dataset -n 240
print("✅ Dataset downloaded!")


## Step 3: Train Tokenizer


In [ ]:
# Train tokenizer on full dataset (GUARANTEED WORKING VERSION)
import os
import torch
import pickle
import tiktoken
from nanochat.tokenizer import HuggingFaceTokenizer, SPECIAL_TOKENS
from nanochat.dataset import parquets_iter_batched

tokenizer_dir = "/root/.cache/nanochat/tokenizer"
os.makedirs(tokenizer_dir, exist_ok=True)

if not os.path.exists(os.path.join(tokenizer_dir, 'tokenizer.pkl')):
    print("="*70)
    print("Training tokenizer on 2B characters...")
    print("Using HuggingFace tokenizer (works on Colab)")
    print("="*70)
    
    # Create text iterator
    def text_iterator():
        nchars = 0
        for batch in parquets_iter_batched(split="train"):
            for doc in batch:
                doc_text = doc
                if len(doc_text) > 10000:  # Cap document length
                    doc_text = doc_text[:10000]
                nchars += len(doc_text)
                yield doc_text
                if nchars > 2000000000:  # 2B characters
                    return
    
    # Train tokenizer
    print("Training BPE tokenizer...")
    hf_tokenizer = HuggingFaceTokenizer.train_from_iterator(text_iterator(), vocab_size=65536)
    
    # Save HuggingFace tokenizer.json
    hf_tokenizer.tokenizer.save(os.path.join(tokenizer_dir, "tokenizer.json"))
    print("✅ Saved tokenizer.json")
    
    # Create tiktoken-compatible encoding for tokenizer.pkl
    vocab = hf_tokenizer.tokenizer.get_vocab()
    
    # Create mergeable_ranks (BPE merges)
    mergeable_ranks = {}
    rank = 0
    for token, token_id in sorted(vocab.items(), key=lambda x: x[1]):
        if token not in SPECIAL_TOKENS:
            try:
                mergeable_ranks[token.encode('utf-8')] = rank
                rank += 1
            except:
                pass
    
    # Create special tokens mapping
    tokens_offset = len(mergeable_ranks)
    special_tokens_dict = {name: tokens_offset + i for i, name in enumerate(SPECIAL_TOKENS)}
    
    # Create tiktoken encoding (what RustBPETokenizer expects)
    enc = tiktoken.Encoding(
        name="huggingface_bpe",
        pat_str=r"""'(?i:[sdmt]|ll|ve|re)|[^\r\n\p{L}\p{N}]?+\p{L}+|\p{N}{1,2}| ?[^\s\p{L}\p{N}]++[\r\n]*|\s*[\r\n]|\s+(?!\S)|\s+""",
        mergeable_ranks=mergeable_ranks,
        special_tokens=special_tokens_dict,
    )
    
    # Save as pickle (what RustBPETokenizer.from_directory expects)
    with open(os.path.join(tokenizer_dir, 'tokenizer.pkl'), 'wb') as f:
        pickle.dump(enc, f)
    print("✅ Saved tokenizer.pkl")
    
    # Create token_bytes.pt (needed for evaluation)
    vocab_size = hf_tokenizer.get_vocab_size()
    special_set = set(hf_tokenizer.get_special_tokens())
    token_bytes = []
    for token_id in range(vocab_size):
        try:
            token_str = hf_tokenizer.decode([token_id])
            if token_str in special_set:
                token_bytes.append(0)
            else:
                token_bytes.append(len(token_str.encode("utf-8")))
        except:
            token_bytes.append(0)
    
    token_bytes = torch.tensor(token_bytes, dtype=torch.int32, device='cpu')
    token_bytes_path = os.path.join(tokenizer_dir, "token_bytes.pt")
    with open(token_bytes_path, "wb") as f:
        torch.save(token_bytes, f)
    print("✅ Saved token_bytes.pt")
    
    print("="*70)
    print("✅ Tokenizer trained and saved successfully!")
    print(f"✅ Files created in: {tokenizer_dir}")
    print(f"   - tokenizer.pkl: {os.path.exists(os.path.join(tokenizer_dir, 'tokenizer.pkl'))}")
    print(f"   - tokenizer.json: {os.path.exists(os.path.join(tokenizer_dir, 'tokenizer.json'))}")
    print(f"   - token_bytes.pt: {os.path.exists(os.path.join(tokenizer_dir, 'token_bytes.pt'))}")
    print("="*70)
else:
    print("✅ Tokenizer already exists, skipping")


## Step 4: Full Model Training

**This trains a production SRGI model:**
- Depth 20 (561M parameters)
- 2048 context length
- Chinchilla-optimal data ratio (20x params)
- Full evaluation suite

**Time**: ~4-8 hours on A100


In [ ]:
# Full production training
print("="*70)
print("🚀 STARTING FULL SRGI MODEL TRAINING")
print("="*70)
print("Model: Depth 20 (561M parameters)")
print("Context: 2048 tokens")
print("Data: Chinchilla-optimal (20x params)")
print("="*70)

!python -m scripts.base_train \
    --depth=20 \
    --max_seq_len=2048 \
    --device_batch_size=32 \
    --total_batch_size=524288 \
    --target_param_data_ratio=20 \
    --run=srgi-production-a100 \
    --eval_tokens=8192 \
    --core_metric_every=1000

print("="*70)
print("✅ TRAINING COMPLETE!")
print("="*70)
